In [1]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

def extract_labels(labels):
    # Map emotion labels to integers
    label_mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    return np.array([label_mapping[label] for label in labels])

class EmotionDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.data, self.labels = self.load_data(dataset_path)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
    @staticmethod
    def load_data(dataset_path):
        data = []
        labels = []

        for emotion_folder in os.listdir(dataset_path):
            emotion_path = os.path.join(dataset_path, emotion_folder)
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                data.append(img)
                labels.append(emotion_folder)

        return np.array(data), extract_labels(labels)
    
    


In [2]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
train_path = 'FER2013/train'
absolute_train_path = os.path.abspath(train_path)

test_path = 'FER2013/test'
absolute_test_path = os.path.abspath(test_path)

train_dataset = EmotionDataset(absolute_train_path, transform=transform)
test_dataset = EmotionDataset(absolute_test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Load pre-trained model

In [4]:
# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda") # Use the GPU for computation
    print("Using GPU for computation!")
else:
    device = torch.device("cpu") # Use the CPU for computation
    print("Using CPU for computation!")

Using GPU for computation!


In [5]:
from torchvision import transforms, models

model = models.resnet18(pretrained=True)

# Modify the final layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 7)  

C:\Users\zyz20\anaconda3\envs\gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zyz20\anaconda3\envs\gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
if torch.cuda.is_available():
    model.to('cuda')
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to('cuda')
else:
    device = torch.device('cpu')
print('Using device:', device)

Using device: cuda


Define Loss function and optimiztion

In [8]:
# Modify the loss function and optimizer to work with GPU
criterion = criterion.to(device)

In [9]:
num_epochs = 30

for epoch in range(num_epochs):

    model.train()
    
    train_accuracy = 0
    num_correct = 0
    num_samples = 0
    
    for inputs, labels in train_loader: 
     
        optimizer.zero_grad()
        
        # Forward pass
        inputs = inputs.to(device) 
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        train_accuracy += acc
        
    # Print epoch loss and accuracy
    avg_acc = train_accuracy / len(train_loader)            
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {avg_acc:.4f}')

Epoch 1/30, Loss: 1.2345, Accuracy: 0.4660
Epoch 2/30, Loss: 1.9866, Accuracy: 0.5966
Epoch 3/30, Loss: 0.4802, Accuracy: 0.6403
Epoch 4/30, Loss: 1.2887, Accuracy: 0.6940
Epoch 5/30, Loss: 1.1554, Accuracy: 0.7484
Epoch 6/30, Loss: 0.4187, Accuracy: 0.8146
Epoch 7/30, Loss: 0.8042, Accuracy: 0.8877
Epoch 8/30, Loss: 0.4048, Accuracy: 0.9273
Epoch 9/30, Loss: 0.0059, Accuracy: 0.9449
Epoch 10/30, Loss: 0.1094, Accuracy: 0.9643
Epoch 11/30, Loss: 0.4074, Accuracy: 0.9632
Epoch 12/30, Loss: 0.0211, Accuracy: 0.9669
Epoch 13/30, Loss: 0.1052, Accuracy: 0.9736
Epoch 14/30, Loss: 0.4611, Accuracy: 0.9754
Epoch 15/30, Loss: 0.0118, Accuracy: 0.9738
Epoch 16/30, Loss: 0.4872, Accuracy: 0.9784
Epoch 17/30, Loss: 0.2120, Accuracy: 0.9650
Epoch 18/30, Loss: 0.0514, Accuracy: 0.9790
Epoch 19/30, Loss: 0.0100, Accuracy: 0.9828
Epoch 20/30, Loss: 0.0003, Accuracy: 0.9806
Epoch 21/30, Loss: 0.0351, Accuracy: 0.9836
Epoch 22/30, Loss: 0.0493, Accuracy: 0.9815
Epoch 23/30, Loss: 0.0664, Accuracy: 0.98

In [10]:
for inputs, labels in train_loader:

    print(type(inputs))
    print(type(labels))
    print(inputs)
    print(labels)
    break

<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2318,  2.2318,  2.2318],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2318,  2.2318,  2.2318],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2318,  2.2318,  2.2318],
          ...,
          [ 1.6153,  1.6153,  1.6153,  ...,  2.2318,  2.2318,  2.2318],
          [ 1.6324,  1.6324,  1.6324,  ...,  2.2318,  2.2318,  2.2318],
          [ 1.6324,  1.6324,  1.6324,  ...,  2.2318,  2.2318,  2.2318]],

         [[ 2.4286,  2.4286,  2.4286,  ...,  2.4111,  2.4111,  2.4111],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4111,  2.4111,  2.4111],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4111,  2.4111,  2.4111],
          ...,
          [ 1.7808,  1.7808,  1.7808,  ...,  2.4111,  2.4111,  2.4111],
          [ 1.7983,  1.7983,  1.7983,  ...,  2.4111,  2.4111,  2.4111],
          [ 1.7983,  1.7983,  1.7983,  ...,  2.4111,  2.4111,  2.4111]],

         [[ 2.6400,  2.6400,  2.6400,  ...,  2.6226,  2.

In [11]:
model.eval()
test_accuracy = 0  
num_correct = 0
num_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        test_accuracy += acc

print(f'Test Accuracy: {test_accuracy / len(test_loader):.4f}')

Test Accuracy: 0.6340


In [13]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_resNet_30.pt') 